<a href="https://colab.research.google.com/github/Dovineowuor/AI-ChatBot/blob/main/HuggingFaceStackUp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quest 3 - Create a Llama 2 Chat Agent

# Learning Outcomes

---

By the end of this quest, you will be able to:

* Set up and configure a chat agent that intelligently integrates a QA dataset with athe Llama 2 model.
* Implement functionality that updates the QA dataset with new entries when an answer is generated by a Llama 2 model.
* Develop an interactive user interface for your chat agent using Gradio, allowing users to interact with it through a web-based platform.
* Understand how to balance between pre-existing knowledge (QA dataset) and AI-generated content in a conversational agent.
* Deploy your chat agent as a web application that becomes more intelligent over time as it learns from new questions and answers.

# Quest Details

---
**Introduction**
In this quest, you will take your skills to the next level by building a dynamic chat agent using the Llama 2 model from Hugging Face Transformers. Unlike a basic chatbot, this chat agent will first check if the question has a predefined answer in a QA dataset, and if not, it will generate a response using the Llama 2 model.

The agent will also automatically update the dataset with new Q&A pairs, ensuring that it becomes more knowledgeable over time. By integrating Gradio, you’ll create an interactive user interface for your chat agent, making it accessible and user-friendly.
This quest will equip you with practical experience in handling both structured (QA dataset) and unstructured (LLM-based responses) data sources, as well as deploying an AI-powered chat service.

For technical help on the StackUp platform & quest-related questions, join our Discord, head to the quest-helpdesk channel and look for the correct thread to ask your question.


**Deliverables**

1. This quest has 1 deliverable.
2. A screenshot


# Hugging Face Setup:
Setup
Configurations and Installations and Running

**Install Gradio Dependencies**

In [ ]:
!pip install accelerate protobuf sentencepiece torch torchvision huggingface_hub git+https://github.com/huggingface/transformers gradio transformers spacy nltk requests optimum[onnxruntime] llama_index pandas aiohttp asyncio


Freeze Package Requirements
```
!pip freeze> requirements.txt
```

In [1]:
!pip freeze> requirements.txt

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(token = userdata.get('HF_TOKEN')) #Hugging Face Token

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline

# Load the pre-trained model and tokenizer from Hugging Face
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

# Create a pipeline for masked language modeling
nlp_pipeline = pipeline("fill-mask", model=model, tokenizer=tokenizer)

# Test the pipeline with a simple input
test_sentence = "The quick brown fox jumps over the [MASK] dog."
result = nlp_pipeline(test_sentence)

print(result)


# Loading The Pre-Trained Language Model Llama 2

In [ ]:
from transformers import (AutoModelForCausalLM,
AutoTokenizer, pipeline)
from huggingface_hub import login
import torch
from google.colab import userdata

# Hugging Face access token 'access-token'
# login(token= userdata.get('HF_TOKEN'), add_to_git_credential=True) # Disable redundant login sessions

model_id = "NousResearch/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False

In [4]:
config = model.config
#Retrieves the configuration of the loaded model,
#which includes details such as the model architecture,
#number of layers, hidden size, etc.

# print(config)

#Outputs a summary of the model architecture,
#showing the various layers and their configurations.
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      



## Load The Sample QA DATA

## A Sample Training Dataset


In [5]:
import pandas as pd

# Sample QA data for Computer Science Theory
qa_data = {
    'question': [
        "What is an algorithm?",
        "What is the difference between a stack and a queue?",
        "What is Big O notation?",
        "Explain the concept of dynamic programming.",
        "What is the purpose of a hash table?",
        "What is a binary tree?",
        "What is a graph in computer science?",
        "Define computational complexity.",
        "What is a sorting algorithm?",
        "Explain the concept of recursion."
    ],
    'answer': [
        "An algorithm is a step-by-step procedure or formula for solving a problem. It is a sequence of instructions that is followed to achieve a desired result.",
        "A stack is a data structure that follows the Last In First Out (LIFO) principle, while a queue follows the First In First Out (FIFO) principle.",
        "Big O notation is used to describe the performance or complexity of an algorithm in terms of time or space. It characterizes algorithms by their worst-case or upper bound performance.",
        "Dynamic programming is a method for solving complex problems by breaking them down into simpler subproblems. It involves storing the results of subproblems to avoid redundant computations.",
        "A hash table is a data structure that maps keys to values for efficient data retrieval. It uses a hash function to compute an index into an array of buckets or slots, from which the desired value can be found.",
        "A binary tree is a data structure in which each node has at most two children, referred to as the left child and the right child. It is used for efficient searching and sorting.",
        "A graph is a collection of nodes (vertices) and edges (connections) that link pairs of nodes. Graphs are used to model relationships between objects.",
        "Computational complexity is a measure of the amount of resources, such as time and space, that an algorithm requires relative to the size of the input data.",
        "A sorting algorithm is a method for arranging elements in a list or array in a specific order, typically ascending or descending. Examples include bubble sort, merge sort, and quicksort.",
        "Recursion is a programming technique where a function calls itself in order to solve a problem. The function typically has a base case to terminate the recursion and a recursive case to break the problem into smaller subproblems."
    ]
}

# Create a DataFrame
df = pd.DataFrame(qa_data)

# Save to CSV
df.to_csv('qa_dataset.csv', index=False)


# Chat System Engine

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from nltk import download
from nltk.corpus import wordnet
import spacy
import gradio as gr

# Download necessary NLTK resources
download('wordnet')

# Load the Llama 2 model and tokenizer
model_name = "NousResearch/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load spaCy model for NER
nlp = spacy.load('en_core_web_sm')

# Load emotion analysis pipeline
emotion_analyzer = pipeline(
    'text-classification',
    model="j-hartmann/emotion-english-distilroberta-base",
    top_k=None,
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

# Initialize cache for responses
response_cache = {}

# Function to analyze emotion
def analyze_emotion(message):
    results = emotion_analyzer(message)
    print("Emotion Analysis Results:", results)  # Debugging line

    # Access the first element of the outer list, then the first dictionary
    if isinstance(results, list) and len(results) > 0 and isinstance(results[0], dict):
        label = results[0].get('label', 'unknown')  # Get the label from the first dictionary
        score = results[0].get('score', 0.0)  # Get the score from the first dictionary
    else:
        label = 'unknown'
        score = 0.0
    return label.lower(), score

# Function to extract keywords from text
def extract_keywords(text):
    doc = nlp(text)
    keywords = []
    for token in doc:
        if token.pos_ in ['NOUN', 'VERB', 'ADJ'] and not token.is_stop and not token.is_punct:
            keywords.append(token.text.lower())
            for syn in wordnet.synsets(token.text):
                for lemma in syn.lemmas():
                    synonyms = lemma.name().lower()
                    if synonyms not in keywords:
                        keywords.append(synonyms)
    return keywords

# Function to generate responses using the model
async def generate_response(message, max_tokens=400, temperature=0.7, top_p=0.9):
    # Encode the input message
    input_ids = tokenizer.encode(message, return_tensors="pt")

    # Move the input tensor to GPU if available, otherwise use CPU
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    input_ids = input_ids.to(device)

    try:
        # Generate a response from the model using the specified parameters
        output = model.generate(
            input_ids,
            max_length=max_tokens,
            num_beams=5,
            no_repeat_ngram_size=2,
            temperature=temperature,
            top_p=top_p
        )
    except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            return "Sorry, the model ran out of memory. Please try with a smaller input or on a different device."
        else:
            raise e

    # Decode the output tokens to generate the final response
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response


# Main chat function
async def chat_function_with_emotions(message, chat_history=[]):
    # Analyze the user's message for emotion
    emotion_label, emotion_score = analyze_emotion(message)
    response = await generate_response(message)

    # Emotion-aware response adjustments
    if emotion_label in ['joy', 'satisfaction']:
        response = f"Glad to hear you're feeling great! 😊 {response}"
    elif emotion_label in ['sadness', 'frustration']:
        response = f"I'm really sorry you're feeling this way. How can I assist you better? 💔 {response}"
    elif emotion_label == 'anger':
        response = f"I sense some frustration. Let’s work through this together! 💪 {response}"
    elif emotion_label == 'fear':
        response = f"It's okay to feel that way. I'm here to help. 🙏 {response}"

    # Update chat history
    chat_history.append((message, response))

    # Cache the response
    response_cache[message] = response
    return "", chat_history

# Define your Gradio Chatbot component
with gr.Blocks() as interface:
    gr.Markdown(
        """
        # Welcome to The Dove Chat Agent! 🌟
        Chat with our intelligent assistant. Type your messages below and receive helpful responses.
        """
    )

    chatbot = gr.Chatbot(
        label="Chat with The Dove",
        value=[
            ("Hello! How can I assist you today?", None),  # Initial bot message
        ],
        elem_id="chatbot"  # CSS ID for additional styling if needed
    )

    with gr.Row():
        text_input = gr.Textbox(show_label=False, placeholder="Type a message and hit enter...")
        text_input.submit(
            chat_function_with_emotions, [text_input, chatbot], [text_input, chatbot]
        )

# Launch the Gradio interface
interface.launch(debug=True)


[nltk_data] Downloading package wordnet to /root/nltk_data...


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1e2a71ec1ee4004125.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Emotion Analysis Results: [[{'label': 'neutral', 'score': 0.7978155016899109}, {'label': 'surprise', 'score': 0.14324189722537994}, {'label': 'fear', 'score': 0.0223916657269001}, {'label': 'anger', 'score': 0.01876799948513508}, {'label': 'disgust', 'score': 0.009277774021029472}, {'label': 'sadness', 'score': 0.005696852225810289}, {'label': 'joy', 'score': 0.0028083384968340397}]]


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


# Import Packages